In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
import nltk

C:\Users\ANIKET RAJ\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1",header=None)
df.head

<bound method NDFrame.head of          0           1                             2         3  \
0        0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1        0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2        0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3        0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4        0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
5        0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY   
6        0  1467811592  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
7        0  1467811594  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
8        0  1467811795  Mon Apr 06 22:20:05 PDT 2009  NO_QUERY   
9        0  1467812025  Mon Apr 06 22:20:09 PDT 2009  NO_QUERY   
10       0  1467812416  Mon Apr 06 22:20:16 PDT 2009  NO_QUERY   
11       0  1467812579  Mon Apr 06 22:20:17 PDT 2009  NO_QUERY   
12       0  1467812723  Mon Apr 06 22:20:19 PDT 2009  NO_QUERY   
13       0  1467812771  Mon Apr 06 22:20:19 PD

In [21]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [22]:
y=df[0]
x=df[5]

#### Without preprocessing

In [31]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X_ngrams = vectorizer.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(
    X_ngrams,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

clf = svm.LinearSVC(loss='hinge')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [32]:
print(metrics.f1_score(y_test, y_pred, pos_label=0))
print(metrics.f1_score(y_test, y_pred, pos_label=4))

0.8056051925403287
0.7985568020566207


In [33]:
print('test set accuracy = ',clf.score(X_test, y_test))
print('training set accuracy = ',clf.score(X_train, y_train))

test set accuracy =  0.80214375
training set accuracy =  0.854765625


In [34]:
print(X_ngrams)

  (0, 588461)	0.3847820406134551
  (0, 282902)	0.13325466639021682
  (0, 627235)	0.1727585123147094
  (0, 155155)	0.14322600378491926
  (0, 12898)	0.4711174549706515
  (0, 86741)	0.21482894077478346
  (0, 600330)	0.10767422927134272
  (0, 123107)	0.25645368289347503
  (0, 675497)	0.09293595022467749
  (0, 552090)	0.30355162901739546
  (0, 256817)	0.13857788944011265
  (0, 176163)	0.24261904027026174
  (0, 132252)	0.35162019379011117
  (0, 458032)	0.10459526434807508
  (0, 607609)	0.27641559689924305
  (0, 177229)	0.12809105297140524
  (0, 613861)	0.07264838063771703
  (0, 192926)	0.13643455387258877
  (0, 300094)	0.09072433436573028
  (1, 600330)	0.1266677119840868
  (1, 300094)	0.10672789518118093
  (1, 299013)	0.11319087135196068
  (1, 635099)	0.280423124045034
  (1, 270301)	0.1853243765194385
  (1, 128986)	0.14739911518782142
  :	:
  (1599997, 402259)	0.15291998788416442
  (1599997, 675991)	0.19572962503227642
  (1599997, 77204)	0.18640791355413427
  (1599997, 507917)	0.264216896803

##### With preprocessing

In [29]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1",header=None)
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [31]:
y=df[0]
x=df[5]
raw_text = x

##### Replace email, URLs, money, phone no. , numbers, collapse all kind of white spaces into single space, convert in lowercase

In [3]:
def preprocess(raw_text):
    processed = raw_text.str.replace(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b',
                                     'emailaddr')
    processed = processed.str.replace(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)',
                                      'httpaddr')
    processed = processed.str.replace(r'£|\$', 'moneysymb')    
    processed = processed.str.replace(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
        'phonenumbr')    
    processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')
    processed = processed.str.replace(r'[^\w\d\s]', ' ')
    processed = processed.str.replace(r'\s+', ' ')
    processed = processed.str.replace(r'^\s+|\s+?$', '')
    processed = processed.str.lower()
    return processed

In [4]:
def stopWordsRemovar(processed):
    stop_words = nltk.corpus.stopwords.words('english')
    processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in set(stop_words)))
    return processed

In [5]:
def stemme(processed):
    porter = nltk.PorterStemmer()
    processed = processed.apply(lambda x: ' '.join(
    porter.stem(term) for term in x.split()))
    return processed

In [37]:
after_preprocess = preprocess(raw_text)
after_stop_words_removal = stopWordsRemovar(after_preprocess)
processed = after_stop_words_removal

In [38]:
processed

0          switchfoot httpaddr awww bummer shoulda got da...
1          upset update facebook texting might cry result...
2          kenichan dived many times ball managed save nu...
3                           whole body feels itchy like fire
4                           nationwideclass behaving mad see
5                                        kwesidei whole crew
6                                                   need hug
7          loltrish hey long time see yes rains bit bit l...
8                                             tatiana_k nope
9                                         twittera que muera
10                           spring break plain city snowing
11                                              pierced ears
12         caregiving bear watch thought ua loss embarras...
13         octolinznumbr counts idk either never talk any...
14         smarrison would first gun really though zac sn...
15         iamjazzyfizzle wish got watch miss iamlilnicki...
16         hollis death 

In [39]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X_ngrams = vectorizer.fit_transform(processed)
X_train, X_test, y_train, y_test = train_test_split(
    X_ngrams,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

clf = svm.LinearSVC(loss='hinge')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [40]:
print(metrics.f1_score(y_test, y_pred, pos_label=0))
print(metrics.f1_score(y_test, y_pred, pos_label=4))

0.7822362908481519
0.7791799099977971


In [41]:
print('test set accuracy = ',clf.score(X_test, y_test))
print('training set accuracy = ',clf.score(X_train, y_train))

test set accuracy =  0.78071875
training set accuracy =  0.84261796875


In [1]:
import xgboost as xgb

In [11]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1",header=None)
df.drop([1,2,3,4],axis=1,inplace=True)
y=df[0]
x=df[5]
raw_text = x

In [12]:
#after_preprocess = preprocess(raw_text)
#after_stop_words_removal = stopWordsRemovar(after_preprocess)
#processed = after_stop_words_removal
processed = raw_text

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X_ngrams = vectorizer.fit_transform(processed)
X_train, X_test, y_train, y_test = train_test_split(
    X_ngrams,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [16]:
xg_reg = xgb.XGBClassifier()
xg_reg.fit(X_train,y_train)

y_pred = xg_reg.predict(X_test)

C:\Users\ANIKET RAJ\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
print(metrics.f1_score(y_test, y_pred, pos_label=0))
print(metrics.f1_score(y_test, y_pred, pos_label=4))

0.6697055570915124
0.7274298425735798


In [20]:
accuracy_score(y_test, y_pred)

0.70133125

In [19]:
from sklearn.metrics import accuracy_score